In [52]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [53]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [54]:
from datetime import datetime
import pandas as pd
from src.data_utils import load_and_process_citibike_data

from_year = 2024
# to_year = datetime.now().year
to_year = 2025
print(f"Download raw data from {from_year} to {to_year}")

rides = pd.DataFrame()
chunks = []
# for year in range(from_year, to_year+1):

#     rides_one_year = load_and_process_citibike_data(year)

#     chunks.append(rides_one_year)
#     #break

for year in range(from_year, to_year+1):
    for month in range(1, 13):
        try:
            rides_one_month = load_and_process_citibike_data(year, months=[month])
            chunks.append(rides_one_month)
        except Exception as e:
            print(f"Skipping {year}-{month:02}: {e}")


# Concatenate all chunks at the end
rides = pd.concat(chunks, ignore_index=True)
print("Data loading complete.")

Download raw data from 2024 to 2025
📁 File already exists: citi_bike_rides_raw_2024_01.parquet
📖 Loading data from: citi_bike_rides_raw_2024_01.parquet
🔍 Filtering data for 2024-01...
✅ Saved filtered data to: /Users/yashmathur/Documents/MS_DS/Python_Spring_25/CDA_500/Test_Final/data/processed/citi_bike_rides_processed_2024_01.parquet
✅ Successfully combined all filtered months.
📁 File already exists: citi_bike_rides_raw_2024_02.parquet
📖 Loading data from: citi_bike_rides_raw_2024_02.parquet
🔍 Filtering data for 2024-02...
✅ Saved filtered data to: /Users/yashmathur/Documents/MS_DS/Python_Spring_25/CDA_500/Test_Final/data/processed/citi_bike_rides_processed_2024_02.parquet
✅ Successfully combined all filtered months.
📁 File already exists: citi_bike_rides_raw_2024_03.parquet
📖 Loading data from: citi_bike_rides_raw_2024_03.parquet
🔍 Filtering data for 2024-03...
✅ Saved filtered data to: /Users/yashmathur/Documents/MS_DS/Python_Spring_25/CDA_500/Test_Final/data/processed/citi_bike_rid

In [55]:
rides

,started_at,start_station_id
0,2024-01-22 18:43:19.012,7954.12
1,2024-01-11 19:19:18.721,6771.13
2,2024-01-30 19:17:41.693,5659.11
3,2024-01-27 11:27:01.759,6771.13
4,2024-01-16 15:15:41.000,7443.01
...,...,...
55247947,2025-04-22 13:19:13.939,6072.11
55247948,2025-04-14 20:10:03.877,6098.1
55247949,2025-04-21 19:59:56.840,5779.08
55247950,2025-04-28 09:12:42.478,6072.11


In [56]:
rides.shape

(55247952, 2)

In [57]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [58]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   hour              34992 non-null  datetime64[ns]
 1   start_station_id  34992 non-null  object        
 2   ride_count        34992 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 820.3+ KB


In [59]:
import hopsworks

api_key = os.getenv('HOPSWORKS_API_KEY')  
project_name = os.getenv('HOPSWORKS_PROJECT_NAME')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")

2025-05-11 12:03:47,195 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:03:47,224 INFO: Initializing external client
2025-05-11 12:03:47,224 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:03:47,773 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Successfully connected to Hopsworks project: Citi_Bike_3010


In [60]:
feature_store = project.get_feature_store()

In [61]:
FEATURE_GROUP_NAME = "citi_bike_time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [62]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time series data at hourly freaquency",
    primary_key=["start_station_id", "hour"],
    event_time="hour"
)

In [63]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 34992/34992 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: citi_bike_time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1231009/jobs/named/citi_bike_time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('citi_bike_time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)